# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import standard libraries
import re 
import pickle 

# import libraries
import nltk
import numpy as np 
import pandas as pd 
from sqlalchemy import create_engine


#nltk downloads 
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

#nltk modules
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# scikit-learn 
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import FeatureUnion, Pipeline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
engine = create_engine('sqlite:///DisasterResponseProject.db')
df = pd.read_sql("SELECT * FROM Message", engine)

In [3]:
# Create a function to load data from database
def load_data(): 
    engine = create_engine('sqlite:///DisasterResponseProject.db')
    df = pd.read_sql("SELECT * FROM Message", engine)
    
    #load and view the text and category variables
    X = df.message.values
    y = df.iloc[:, 4:].values
    
    #get categories/column names
    categories = df.columns[4:].tolist()
    
    return X, y, categories

In [4]:
X, y, categories= load_data()
X.shape, y.shape

((26216,), (26216, 35))

### 2. Write a tokenization function to process your text data

In [5]:
#Create a function that normalizes, tokenizes and lemmatizes the messages
def tokenize(text):
    #remove punctuation such as Twitter(#) and tags(@) and convert to lowercase
    text = re.sub(r"[^a-zA-Z0-9]", " ", text).lower()
    
    #Split text into words using NLTK
    tokens = word_tokenize(text) 
    
    #Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    #Remove stopwords and lemmatize
    clean_tokens = [
        lemmatizer.lemmatize(w) for w in tokens if w not in stopwords.words("english")
    ]
    
    return clean_tokens

In [6]:
#test out function
for message in X[:5]:
    tokens = tokenize(message)
    print("Original message:", message)
    print("Tokenized:", tokens, "\n")

Original message: Weather update - a cold front from Cuba that could pass over Haiti
Tokenized: ['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Original message: Is the Hurricane over or is it not over
Tokenized: ['hurricane'] 

Original message: Looking for someone but no name
Tokenized: ['looking', 'someone', 'name'] 

Original message: UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
Tokenized: ['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

Original message: says: west side of Haiti, rest of the country today and tonight
Tokenized: ['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
def ml_pipeline(): 
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)), 
        ('tfidf', TfidfTransformer()), 
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
#Split tada into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
#train classifier
pipeline = ml_pipeline()
pipeline.fit(X_train, y_train)

#predict on test data (transofrm test data)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
print(classification_report(y_test, y_pred, target_names=categories))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      5992
               request       0.78      0.43      0.55      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.74      0.62      0.67      3219
          medical_help       0.57      0.09      0.16       638
      medical_products       0.79      0.09      0.16       418
     search_and_rescue       0.60      0.08      0.14       192
              security       0.00      0.00      0.00       144
              military       0.44      0.13      0.21       245
                 water       0.85      0.33      0.47       500
                  food       0.81      0.35      0.49       878
               shelter       0.80      0.30      0.44       705
              clothing       0.80      0.10      0.18       115
                 money       0.70      0.04      0.08       170
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Based on the results of our first model, we can see that: 
- there is a 0.73 accuracy in positive predictions
- the recall is 0.48, which means that the model is capturing half of the actual positive cases. 
- the F1 score is 0.54, which shows that there is potential for improvement. 

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7e0b6f0c5a60>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [12]:
def ml_cv_pipeline(): 
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 1))), 
        ('tfidf', TfidfTransformer()), 
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
    ])
    
    # Specify the parameters for grid search 
    parameters = {
        # Try different n-gram ranges (1 word)
        'vect__ngram_range': [(1, 1)],
        # Try different number of trees in the random forest 
        'clf__estimator__n_estimators': [5, 10]
    }
    
    # Create a grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, n_jobs=-1)
    return cv

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
#train classifier
ml_cv = ml_cv_pipeline()
ml_cv.fit(X_train, y_train)

#Get the best parameters
print('Best parameters found:', ml_cv.best_params_)

#predict on test data (transform test data)
y_pred = ml_cv.predict(X_test)

Best parameters found: {'clf__estimator__n_estimators': 10, 'vect__ngram_range': (1, 1)}


In [14]:
print(classification_report(y_test, y_pred, target_names=categories))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      5992
               request       0.79      0.42      0.55      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.74      0.62      0.67      3219
          medical_help       0.61      0.10      0.17       638
      medical_products       0.68      0.07      0.12       418
     search_and_rescue       0.60      0.06      0.11       192
              security       0.00      0.00      0.00       144
              military       0.53      0.09      0.16       245
                 water       0.83      0.22      0.34       500
                  food       0.86      0.41      0.55       878
               shelter       0.79      0.31      0.44       705
              clothing       0.75      0.10      0.18       115
                 money       0.86      0.07      0.13       170
        missing_people       1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


When adding a GridSearchCV with parameters, we could see a slight improvement (1%) on recalls. However, due to this model being highly computationally extensive, we could not work using several parameters. So we can see further room for improvement. 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
def ml_fu_pipeline_2():
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipelline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)), 
                ('tfidf', TfidfTransformer())
            ])),
        ])), 
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    return pipeline

In [16]:
#train classifier
ml_fu = ml_fu_pipeline_2()
ml_fu.fit(X_train, y_train)

#predict on test data (transform test data)
y_pred = ml_fu.predict(X_test)

In [17]:
# Print classification report
print(classification_report(y_test, y_pred, target_names=categories))

                        precision    recall  f1-score   support

               related       0.80      0.96      0.87      5992
               request       0.75      0.49      0.60      1332
                 offer       0.08      0.03      0.04        36
           aid_related       0.75      0.60      0.67      3219
          medical_help       0.60      0.27      0.37       638
      medical_products       0.66      0.31      0.42       418
     search_and_rescue       0.57      0.18      0.27       192
              security       0.19      0.03      0.06       144
              military       0.57      0.30      0.39       245
                 water       0.75      0.64      0.69       500
                  food       0.82      0.73      0.77       878
               shelter       0.77      0.56      0.65       705
              clothing       0.69      0.44      0.54       115
                 money       0.45      0.28      0.35       170
        missing_people       0.59      

Adding some flexibility to our model with FeatureUnion and changing RandomForestClassifier() to AdaBoostClassifier() instead, to learn from our mistakes, allowed us some improvement. 
- Though the precision declined by 1%, the recall improved by 10% compared to the first model. 
- Therefore, our F1 score increased to 0.62. 

This means that we should keep this model. Further improvement could be also be built on this direction. 

### 9. Export your model as a pickle file

In order to meet the rubric from this project, I will save the ml_cv model to a pickle file and continue working with this model throughout the project, as it includes GridSearchCV. 

In [18]:
# Save the model to a pickle file
pickle_model = 'pickle_model.pkl'
with open(pickle_model, 'wb') as file:
    pickle.dump(ml_cv, file)

print(f'Model saved to {pickle_model}')

Model saved to pickle_model.pkl


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.